In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [35]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Type casting

In [50]:
float_columns = ['event_participation_rate', 'avg_session_length', 'sessions_per_week', 'total_playtime_hours', 'days_since_last_login',
'historical_spending', 'prev_month_spending', 'avg_transaction_value', 'days_since_last_purchase', 'purchase_frequency']

cat_columns = ['guild_membership', 'is_premium_member', 'primary_game', 'platform', 'owns_limited_edition', 'tournament_participation', 'segment']

In [51]:
# Cast categorical columns
for col in cat_columns:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Cast float columns (ไว้เป็น float)
for col in float_columns:
    train_df[col] = train_df[col].astype('float64')
    test_df[col] = test_df[col].astype('float64')

# Cast ที่เหลือเป็น int
all_columns = set(train_df.select_dtypes(include=['float64']).columns)
exclude_columns = set(float_columns + cat_columns + ['spending_30d'])  # เผื่อ target เป็น float
int_columns = list(all_columns - exclude_columns)

for col in int_columns:
    train_df[col] = train_df[col].fillna(-1).astype('int64')
    test_df[col] = test_df[col].fillna(-1).astype('int64')

# Prerpocessing

## Handle Missing Values

In [52]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

iter_imputer = IterativeImputer(max_iter=10, random_state=2025)

# Initial
all_num_cols = float_columns + int_columns
special_columns = [col for col in all_num_cols if train_df[col].min() == -1]
train_df[special_columns] = train_df[special_columns].replace(-1, np.nan)
test_df[special_columns] = test_df[special_columns].replace(-1, np.nan)

# Fit and transform
train_df[all_num_cols] = iter_imputer.fit_transform(train_df[all_num_cols])
test_df[all_num_cols] = iter_imputer.transform(test_df[all_num_cols])

# categorical column
cat_imputer = SimpleImputer(strategy='most_frequent')
train_df[cat_columns] = cat_imputer.fit_transform(train_df[cat_columns])
test_df[cat_columns] = cat_imputer.transform(test_df[cat_columns])
train_df[cat_columns] = train_df[cat_columns].astype('category')
test_df[cat_columns] = test_df[cat_columns].astype('category')

In [53]:
print(train_df.isnull().sum().sum())
print(test_df.isnull().sum().sum())

0
0


# Feature Engineering

In [54]:
def feature_engineering(df):
    df_ = df.copy()

    # ========== Log Transform ==========
    df_['historical_spending_log'] = np.log1p(df_['historical_spending'])
    df_['prev_month_spending_log'] = np.log1p(df_['prev_month_spending'])
    df_['avg_transaction_value_log'] = np.log1p(df_['avg_transaction_value'])
    df_['total_playtime_hours_log'] = np.log1p(df_['total_playtime_hours'])

    # ========== Cyclical Encoding ==========
    df_['seasonal_sin'] = np.sin(df_['seasonal_spending_pattern'] * (2 * np.pi / 12))
    df_['seasonal_cos'] = np.cos(df_['seasonal_spending_pattern'] * (2 * np.pi / 12))

    # ========== Spending Behavior Features ==========
    # Average spending per transaction
    df_['spend_per_transaction'] = df_['historical_spending'] / (df_['total_transactions'] + 1)
    
    # Spending momentum (prev month vs historical average)
    df_['spending_trend'] = df_['prev_month_spending'] / (df_['historical_spending'] / 12 + 1)
    
    # Purchase recency score (inverse of days since last purchase)
    df_['purchase_recency'] = 1 / (df_['days_since_last_purchase'] + 1)
    
    # Discount affinity (how much they use discounts)
    df_['discount_affinity'] = df_['purchases_on_discount'] / (df_['total_transactions'] + 1)

    # Purchase velocity (change in spending rate)
    df_['purchase_velocity'] = (
        df_['prev_month_spending'] - (df_['historical_spending'] / 12)
    ) / (df_['historical_spending'] / 12 + 1)

    # Spending acceleration (change in spending relative to avg transaction)
    df_['spending_acceleration'] = (
        df_['prev_month_spending'] - df_['avg_transaction_value']
    ) / (df_['avg_transaction_value'] + 1)
    
    # ========== Engagement Features ==========
    # Session intensity (playtime per session)
    df_['session_intensity'] = df_['total_playtime_hours'] / (df_['sessions_per_week'] * 52 + 1)
    
    # Activity recency (inverse of days since last login)
    df_['login_recency'] = 1 / (df_['days_since_last_login'] + 1)
    
    # Engagement consistency (login streak relative to account age)
    df_['engagement_consistency'] = df_['daily_login_streak'] / (df_['account_age_days'] + 1)

    # Engagement decay (days since last login relative to login streak)
    df_['engagement_decay'] = df_['days_since_last_login'] / (df_['daily_login_streak'] + 1)

    df_['total_activity_score'] = (
        df_['sessions_per_week'] * 
        df_['avg_session_length'] * 
        df_['daily_login_streak']
    )
    
    df_['social_activity_ratio'] = (
        df_['social_interactions'] / 
        (df_['sessions_per_week'] * 52 + 1)
    )

    # ========== Social Features ==========
    # Social engagement rate
    df_['social_engagement'] = df_['social_interactions'] / (df_['friend_count'] + 1)

    # ========== Achievement Features ==========
    # Achievement velocity (achievements per day of account age)
    df_['achievement_velocity'] = df_['achievement_count'] / (df_['account_age_days'] + 1)

    # ========== Cross-Game Features ==========
    # Game diversity (games played relative to cross-game activity)
    df_['game_diversity'] = df_['games_played'] / (df_['cross_game_activity'] + 1)

    # ========== Transaction Patterns ==========
    # Transaction frequency score
    df_['transaction_frequency_score'] = df_['purchase_frequency'] * df_['total_transactions']
    
    # High value spender flag (above 75th percentile in avg transaction)
    df_['is_high_value_spender'] = (
        df_['avg_transaction_value'] > df_['avg_transaction_value'].quantile(0.75)
    ).astype(int).astype('category')

    # FIX: Convert category to numeric before arithmetic operations
    df_['premium_engagement'] = (
        (df_['is_premium_member'].cat.codes + df_['vip_status']) * 
        df_['avg_session_length']
    )
    
    df_['value_consistency'] = df_['avg_transaction_value'] / (df_['historical_spending'] / (df_['total_transactions'] + 1))

    # ========== Interaction Features ==========
    # Engagement × Spending (active spenders)
    df_['engagement_x_spending'] = (
        (df_['sessions_per_week'] * df_['avg_session_length']) * 
        df_['prev_month_spending_log']
    )
    
    # Social × Spending (social spenders)
    df_['social_x_spending'] = df_['social_interactions'] * df_['prev_month_spending_log']
    
    # Event participation × VIP (premium event participants)
    df_['event_x_vip'] = df_['event_participation_rate'] * df_['vip_status']

    # Behavioral Segments
    df_['whale_potential'] = (
        df_['vip_status'] * 
        np.log1p(df_['avg_transaction_value']) * 
        df_['purchase_frequency']
    )

    # Churn Risk Score
    df_['churn_risk_score'] = (
        df_['days_since_last_login'] * 
        df_['days_since_last_purchase'] / 
        (df_['daily_login_streak'] + 1)
    )

    # ====== Efficiency Ratios ======
    df_['achievement_per_playtime'] = df_['achievement_count'] / (df_['total_playtime_hours'] + 1)
    df_['social_to_friend_ratio'] = df_['social_interactions'] / (df_['friend_count'] + 1)
    df_['discount_dependency'] = df_['purchases_on_discount'] / (df_['total_transactions'] + 1)
    df_['playtime_per_session'] = df_['total_playtime_hours'] / (df_['sessions_per_week'] * 52 + 1)

    # ====== Binning Features ======
    df_['account_age_bin'] = pd.cut(
        df_['account_age_days'], 
        bins=[0, 180, 365, 730, np.inf],
        labels=['new', 'medium', 'veteran', 'ancient'],
        include_lowest=True
    ).astype('category')

    df_['playtime_bin'] = pd.cut(
        df_['total_playtime_hours'],
        bins=[0, 100, 500, 1000, np.inf],
        labels=['casual', 'regular', 'hardcore', 'extreme'],
        include_lowest=True
    ).astype('category')

    return df_

train_feat_df = feature_engineering(train_df)
test_feat_df = feature_engineering(test_df)

/opt/anaconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encoun

In [55]:
print(train_feat_df.isnull().sum().sum())
print(train_feat_df.isnull().sum().sum())

3108
3108


In [56]:
train_feat_df.columns[train_feat_df.isnull().any()]

Index(['historical_spending_log', 'prev_month_spending_log',
       'avg_transaction_value_log', 'discount_affinity',
       'engagement_x_spending', 'social_x_spending', 'whale_potential',
       'discount_dependency'],
      dtype='object')

# Encode Categorical Features

In [42]:
cat_columns = train_feat_df.select_dtypes(include=['category']).columns

def encode_categorical(df):
    df_ = df.copy()
    # One-hot encode categorical features
    df_ = pd.get_dummies(df_, columns=cat_columns, drop_first=True)
    return df_

train_encode_df = encode_categorical(train_feat_df)
test_encode_df = encode_categorical(test_feat_df)

In [43]:
test_encode_df.head()

,id,player_id,friend_count,social_interactions,event_participation_rate,daily_login_streak,avg_session_length,sessions_per_week,total_playtime_hours,days_since_last_login,...,segment_1.0,segment_2.0,segment_3.0,is_high_value_spender_1,account_age_bin_medium,account_age_bin_veteran,account_age_bin_ancient,playtime_bin_regular,playtime_bin_hardcore,playtime_bin_extreme
0,ANS00001,P128956,177.0,71.0,33.157419,138.0,143.414888,7.151850,1051.879412,10.598046,...,True,False,False,False,True,False,False,False,False,True
1,ANS00002,P115227,55.0,99.0,27.683671,54.0,253.827162,31.460513,433.186064,4.108231,...,True,False,False,False,False,False,True,True,False,False
2,ANS00003,P013087,28.0,69.0,75.184955,101.0,114.896217,34.737910,669.426704,0.015370,...,False,False,True,True,False,True,False,False,True,False
3,ANS00004,P045326,79.0,78.0,17.440872,15.0,119.473134,12.991553,729.958232,0.616083,...,False,False,False,False,True,False,False,False,True,False
4,ANS00005,P052078,121.0,84.0,52.693562,4.0,176.099045,12.882396,520.605664,3.948152,...,False,True,False,False,False,False,True,False,True,False


In [47]:
print(test_encode_df.isnull().sum().sum())
print(test_encode_df.isnull().sum().sum())

851
851


# Scale Features

In [45]:
from sklearn.preprocessing import RobustScaler

target_column = 'spending_30d'
float_columns = train_encode_df.select_dtypes(include=['float64']).columns.difference([target_column])

scaler = RobustScaler()

train_scale_df = train_encode_df.copy()
test_scale_df = test_encode_df.copy()
train_scale_df[float_columns] = scaler.fit_transform(train_encode_df[float_columns])
test_scale_df[float_columns] = scaler.transform(test_encode_df[float_columns])

In [46]:
test_scale_df.shape

(25889, 80)

# Feature Selection

In [31]:
def remove_high_correlation(df, threshold=0.9, target_col='spending_30d'):
    # Get numeric columns only (exclude target and IDs)
    exclude_cols = [target_col, 'id', 'player_id']
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    numeric_cols = [col for col in numeric_cols if col not in exclude_cols]
    
    # Calculate correlation matrix
    corr_matrix = df[numeric_cols].corr().abs()
    
    # Get upper triangle (to avoid duplicates)
    upper_triangle = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )
    
    # Find features with correlation > threshold
    to_drop = []
    for column in upper_triangle.columns:
        # Find correlations above threshold
        correlated_features = upper_triangle[column][upper_triangle[column] > threshold]
        
        if len(correlated_features) > 0:
            # Check correlation with target to decide which to keep
            for corr_feature in correlated_features.index:
                if corr_feature not in to_drop and column not in to_drop:
                    # Keep the one with higher correlation to target
                    corr_col_target = abs(df[column].corr(df[target_col]))
                    corr_feat_target = abs(df[corr_feature].corr(df[target_col]))
                    
                    if corr_col_target >= corr_feat_target:
                        to_drop.append(corr_feature)
                        print(f"Drop '{corr_feature}' (corr with '{column}': {upper_triangle.loc[corr_feature, column]:.3f})")
                    else:
                        to_drop.append(column)
                        print(f"Drop '{column}' (corr with '{corr_feature}': {upper_triangle.loc[corr_feature, column]:.3f})")
                        break
    
    # Remove duplicates
    to_drop = list(set(to_drop))
    
    # Drop features
    df_reduced = df.drop(columns=to_drop)
    
    return df_reduced, to_drop

train_reduced, dropped_features = remove_high_correlation(train_scale_df, threshold=0.95, target_col='spending_30d')
test_reduced = test_scale_df.drop(columns=dropped_features)

print(f"\nOriginal features: {train_scale_df.shape[1]}")
print(f"Reduced features Train: {train_reduced.shape[1]}")
print(f"Reduced features Test: {test_reduced.shape[1]}")
print(f"Features removed: {len(dropped_features)}")

Drop 'prev_month_spending' (corr with 'historical_spending': 0.996)
Drop 'historical_spending_log' (corr with 'prev_month_spending_log': 0.998)
Drop 'purchase_velocity' (corr with 'spending_trend': 1.000)
Drop 'social_engagement' (corr with 'social_to_friend_ratio': 1.000)
Drop 'discount_affinity' (corr with 'discount_dependency': 1.000)
Drop 'session_intensity' (corr with 'playtime_per_session': 1.000)

Original features: 81
Reduced features Train: 75
Reduced features Test: 74
Features removed: 6


In [32]:
train_final = train_reduced.copy()
test_final = test_reduced.copy()

train_final.to_csv("train_preprocessed_v4.csv", index=False)
test_final.to_csv("test_preprocessed_v4.csv", index=False)

In [33]:
target_col = 'spending_30d'
corr_with_target = train_reduced.drop(columns=['id', 'player_id']).corr()[target_col].drop(target_col).abs()
top_10_corr = corr_with_target.sort_values(ascending=False).head(10)
print(f"Top 10 features correlated with '{target_col}':")
for idx, (feature, corr_value) in enumerate(top_10_corr.items(), 1):
    print(f"{idx}. '{feature}' (corr={corr_value:.3f})")

Top 10 features correlated with 'spending_30d':
1. 'historical_spending' (corr=0.866)
2. 'segment_3.0' (corr=0.727)
3. 'prev_month_spending_log' (corr=0.629)
4. 'whale_potential' (corr=0.579)
5. 'event_x_vip' (corr=0.579)
6. 'is_high_value_spender_1' (corr=0.533)
7. 'avg_transaction_value_log' (corr=0.513)
8. 'event_participation_rate' (corr=0.506)
9. 'vip_status' (corr=0.452)
10. 'premium_engagement' (corr=0.435)
